In [15]:
import boto3

s3 = boto3.resource('s3')

# list all buckets
for bucket in s3.buckets.all():
    print(bucket.name)

classgpt
hackathonfiles123
kafka-stock-market-project-ben


In [18]:
# select bucket by name
bucket = s3.Bucket('classgpt')

class_name = "COMS472"
bucket.put_object(Key=f'{class_name}/')

s3.Object(bucket_name='classgpt', key='COMS472/')

In [19]:
def folder_exists(bucket, folder_name):
    for _ in bucket.objects.filter(Prefix=f"{folder_name}/"):
        return True
    return False
    
folder_exists(bucket, class_name)

True

In [36]:
# select bucket by name
bucket = s3.Bucket("classgpt")
class_name = "COMS472"

if not folder_exists(bucket, class_name):
    # create folder
    bucket.put_object(Key=f"{class_name}/")

# upload file
file_path = "pdfs/lecture01-intro-2up.pdf"
file_name = file_path.split("/")[-1]
bucket.upload_file(file_path, f"{class_name}/{file_name}")

file_path = "pdfs/lecture02-agents-2up.pdf"
file_name = file_path.split("/")[-1]
bucket.upload_file(file_path, f"{class_name}/{file_name}")

In [37]:
# list files in folder folder
for obj in bucket.objects.filter(Prefix=f"{class_name}/"):
    print(obj.key)

COMS472/
COMS472/lecture01-intro-2up.pdf
COMS472/lecture02-agents-2up.pdf


In [22]:
# select bucket by name
bucket = s3.Bucket("classgpt")
class_name = "STAT474"

if not folder_exists(bucket, class_name):
    # create folder
    bucket.put_object(Key=f"{class_name}/")

# upload file
file_path = "pdfs/lecture01-intro-2up.pdf"
file_name = file_path.split("/")[-1]
bucket.upload_file(file_path, f"{class_name}/{file_name}")

In [23]:
# list files in folder folder
for obj in bucket.objects.filter(Prefix=""):
    print(obj.key)

COMS472/
COMS472/lecture01-intro-2up.pdf
STAT474/
STAT474/lecture01-intro-2up.pdf


In [24]:
# list folders with s3 resource in bucket
for obj in bucket.objects.filter(Prefix=""):
    if obj.key.endswith("/"):
        print(obj.key)

COMS472/
STAT474/


In [33]:
from pprint import pprint

classes = {}

for obj in bucket.objects.all():
    if obj.key.endswith("/"):
        classes[obj.key[:-1]] = []
    else:
        cname, fname = obj.key.split("/")
        classes[cname].append(fname)

pprint(classes)

{'COMS472': ['lecture01-intro-2up.pdf'], 'STAT474': ['lecture01-intro-2up.pdf']}


In [29]:
# remove all objects in a bucket
bucket = s3.Bucket("classgpt")
bucket.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'YX3CFM8P2H020NZ4',
   'HostId': 'JZa9Ua2VRbkzogE1bVSGsC60UTPFd9jSfVanL0DD+jV7ZnbNfwn6PjdhnibgJuroxjqNv9MwuuU=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'JZa9Ua2VRbkzogE1bVSGsC60UTPFd9jSfVanL0DD+jV7ZnbNfwn6PjdhnibgJuroxjqNv9MwuuU=',
    'x-amz-request-id': 'YX3CFM8P2H020NZ4',
    'date': 'Thu, 02 Mar 2023 17:03:46 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'COMS472/'},
   {'Key': 'STAT474/lecture01-intro-2up.pdf'},
   {'Key': 'COMS472/lecture01-intro-2up.pdf'},
   {'Key': 'STAT474/'}]}]